In [80]:
import numpy as np
import pandas as pd
import random

In [81]:
# new data to combine with initial data (Note: formatted differently from df_2018 data)
df_ks = pd.read_csv("../Data/kickstarter.csv")

In [82]:
# Cleaning
# 1. Delete the first column (i.e., the row id 'Unnamed')
del df_ks['Unnamed: 0']

In [83]:
# 2. Replace state string with numbers in df_sk
cleanup_nums = {"state": {"failed": 0, "successful": 1, "canceled" : -1, "undefined" : -1, "live" : -1, "suspended" : -1}}
df_ks.replace(cleanup_nums, inplace=True)

In [84]:
# 3. Remove Duplicate values
# 3a. Sort by the number of backers (assuming you'll have more backers)
df_ks_sorted = df_ks.sort_values(by='backers_count')
# 3b. Remove duplicates
df_ks_sorted.drop_duplicates(subset='id', keep='last', inplace=True)

In [87]:
# 4. Create new dataframe with the columns (that are not in 2018) = 
# id, backers_count, blurb, blurb_length, spotlight, staff_pick, location_state, location_displayable_name
df_ks_dropped = df_ks_sorted[['id','backers_count', 'blurb', 'blurb_length', 'spotlight', 'staff_pick', 'location_state', 'location_displayable_name']].sort_values(by='id')

In [95]:
df_ks_dropped.head()

,id,backers_count,blurb,blurb_length,spotlight,staff_pick,location_state,location_displayable_name
343901,8624,737,A compact outdoor cooking set that you can use...,12,True,True,WV,"Belgium, WV"
212211,18520,4,Raising money to help my grandmother recover f...,24,False,False,OH,"Cleveland, OH"
11159,21109,11,My work is performance based but I branch out ...,24,True,False,Scotland,"Dundee, UK"
62717,24380,18,A sanctuary for humans and felines alike! Come...,24,False,False,IL,"Bloomington, IL"
34144,33867,42,Taste Makers is a socially conscious brand tha...,23,False,True,BC,"Vancouver, Canada"


In [96]:
# Read in other Data files
df_2018 = pd.read_csv("../Data/Cleaned/2018.csv")
# Rename ID to id
df_2018 = df_2018.rename(columns={"ID": "id"})
df_2018.head(3)

,id,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,0,0,GB,0.0,0.0,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,0,15,US,100.0,2421.0,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,0,3,US,220.0,220.0,45000.00


In [91]:
# Join the two tables on id
df_joined_ids = df_ks_dropped.set_index('id').join(df_2018.set_index('id'))

In [100]:
# Remove any rows that have a NaN in the name column
df_joined_ids = df_joined_ids.dropna();
df_joined_ids.shape

(242630, 21)

In [101]:
df_joined_ids.to_csv("../Data/Cleaned/joined_data.csv", index = False)